In [1]:
import keras
import os
import numpy as np
import edward as ed
import tensorflow as tf
import matplotlib.pyplot as plt

from edward.models import Normal
from functools import partial
    
class DefaultModel:
    def __init__(self):
        pass
        
    def train(self):
        pass
    
class BayesianModel:
    def neural_network(self, x, W_0, W_1, b_0, b_1):
        h = tf.tanh(tf.matmul(x, W_0) + b_0)
        h = tf.matmul(h, W_1) + b_1
        return h

    def __init__(self, features, output):
        middle = output*2
        W_0 = Normal(loc=tf.zeros([features, middle]), scale=tf.ones([features, middle]))
        W_1 = Normal(loc=tf.zeros([middle, output]), scale=tf.ones([middle, output]))
        b_0 = Normal(loc=tf.zeros(middle), scale=tf.ones(middle))
        b_1 = Normal(loc=tf.zeros(output), scale=tf.ones(output))

        self.x = tf.placeholder(tf.float32, shape=(None,features))
        self.y = Normal(loc=self.neural_network(self.x, W_0, W_1, b_0, b_1),
                   scale=0.1 * tf.ones(output))

        qW_0 = Normal(loc=tf.get_variable("qW_0/loc", [features, middle]),
                      scale=tf.nn.softplus(tf.get_variable("qW_0/scale", [features, middle])))
        qW_1 = Normal(loc=tf.get_variable("qW_1/loc", [middle, output]),
                      scale=tf.nn.softplus(tf.get_variable("qW_1/scale", [middle, output])))
        qb_0 = Normal(loc=tf.get_variable("qb_0/loc", [middle]),
                      scale=tf.nn.softplus(tf.get_variable("qb_0/scale", [middle])))
        qb_1 = Normal(loc=tf.get_variable("qb_1/loc", [output]),
                      scale=tf.nn.softplus(tf.get_variable("qb_1/scale", [output])))
       
        self.mus = tf.stack(
            [self.neural_network(self.x, qW_0.sample(), qW_1.sample(),
                                 qb_0.sample(), qb_1.sample())
             for _ in range(10)])
        
        self.sess = ed.get_session()
#         writer = tf.summary.FileWriter("graph")
#         writer.add_graph(sess.graph)
#         tf.global_variables_initializer().run()
        self.inference_partial = partial(ed.KLqp,
                                         latent_vars={W_0: qW_0, b_0: qb_0, W_1: qW_1, b_1: qb_1})
        
    def train(self, x_train, y_train):
        self.inference = self.inference_partial(data={self.x:x_train, self.y: y_train})
        self.inference.run(n_iter=1000, n_samples=5)
    
    def evaluate(self, x_data, n=1) :
        outputs = []
        for i in range(0, n) :
            outputs.append(self.mus.eval(feed_dict={self.x:x_data}))
        
        return np.array(outputs)
        

/home/tryan/miniconda2/envs/stocklive/lib/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)
Using TensorFlow backend.
